# Deep Features for image classification and retrieval Assignment

In this module, we focused on using deep learning to create non-linear features to improve the performance of machine learning. We also saw how transfer learning techniques can be applied to use deep features learned with one dataset to get great performance on a different dataset. We also built an iPython notebooks for both image retrieval and image classification tasks on real datasets.

In this assignment, we are going to build new image retrieval models and explore their results on different parts of our image dataset. These techniques will be used at the core of the intelligent application in your capstone project. 

In [2]:
import graphlab
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

This non-commercial license of GraphLab Create for academic use is assigned to miguelfzafra@gmail.com and will expire on July 28, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1533388156.log


Importamos los elementos de las sesiones de teoría:

In [70]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

graphlab.canvas.set_target('ipynb')

def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

## 1. Computing summary statistics of the data

 Using the training data, compute the sketch summary of the ‘label’ column and interpret the results. What’s the least common category in the training data?

In [10]:
image_train.groupby(key_columns='label',
                    operations = {'count':graphlab.aggregate.COUNT('label')})

Columns:
	label	str
	count	int

Rows: 4

Data:
+------------+-------+
|   label    | count |
+------------+-------+
|    dog     |  509  |
|    cat     |  509  |
|    bird    |  478  |
| automobile |  509  |
+------------+-------+
[4 rows x 2 columns]

In [29]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


## 2.  Creating category-specific image retrieval models

In most retrieval tasks, the data we have is unlabeled, thus we call these unsupervised learning problems. However, we have labels in this image dataset, and will use these to create one model for each of the 4 image categories, {‘dog’,’cat’,’automobile’,bird’}. To start, follow these steps:

Split the SFrame with the training data into 4 different SFrames. Each of these will contain data for 1 of the 4 categories above. Hint: if you use a logical filter to select the rows where the ‘label’ column equals ‘dog’, you can create an SFrame with only the data for images labeled ‘dog’.Similarly to the image retrieval notebook you downloaded, you are going to create a nearest neighbor model using the 'deep_features' as the features, but this time create one such model for each category, using the corresponding subset of the training_data. You can call the model with the ‘dog’ data the dog_model, the one with the ‘cat’ data the cat_model, as so on.

You now have a nearest neighbors model that can find the nearest ‘dog’ to any image you give it, the dog_model; one that can find the nearest ‘cat’, the cat_model; and so on. 

In [32]:
# Dividimos el dataset

dog_train = image_train[image_train['label'] == 'dog']
cat_train = image_train[image_train['label'] == 'cat']
bird_train = image_train[image_train['label'] == 'bird']
automobile_train = image_train[image_train['label'] == 'automobile']

In [33]:
# Entrenamos los modelos

dog_model = graphlab.nearest_neighbors.create(dog_train,features=['deep_features'],
                                             label='id')

cat_model = graphlab.nearest_neighbors.create(cat_train,features=['deep_features'],
                                             label='id')

bird_model = graphlab.nearest_neighbors.create(bird_train,features=['deep_features'],
                                             label='id')

automobile_model = graphlab.nearest_neighbors.create(automobile_train,features=['deep_features'],
                                             label='id')


Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

 The cat image below is the first in the test data:

In [79]:
image_test[0:1]['image'].show()

What is the nearest ‘cat’ labeled image in the training data to the cat image above (the first image in the test data)?

In [35]:
cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 32.354ms     |

| Done         |         | 100         | 238.717ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [82]:
cat_train[cat_train['id'] == 16289]['image'].show()

What is the nearest ‘dog’ labeled image in the training data to the cat image above (the first image in the test data)?

In [36]:
dog_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 33.333ms     |

| Done         |         | 100         | 280.279ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [83]:
dog_train[dog_train['id'] == 16976]['image'].show()

## 3. . A simple example of nearest-neighbors classification

When we queried a nearest neighbors model, the ‘distance’ column in the table above shows the computed distance between the input and each of the retrieved neighbors. In this question, you will use these distances to perform a classification task, using the idea of a nearest-neighbors classifier.

For the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘cat’ in the training data (similarly to what you did in the previous question).

In [37]:
# Distancia media a gatos
cat_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 41.737ms     |

| Done         |         | 100         | 306.313ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

Similarly, for the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘dog’ in the training data (similarly to what you did in the previous question). 

In [38]:
dog_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 32.723ms     |

| Done         |         | 100         | 256.793ms    |

+--------------+---------+-------------+--------------+

37.77071136184157

On average, is the first image in the test data closer to its 5 nearest neighbors in the ‘cat’ data or in the ‘dog’ data? (In a later course, we will see that this is an example of what is called a k-nearest neighbors classifier, where we use the label of neighboring points to predict the label of a test point.)

Vemos que es menor la distancia a los gatos que a los perros (tiene sentido).

## 4.  Computing nearest neighbors accuracy using SFrame operations

 A nearest neighbor classifier predicts the label of a point as the most common label of its nearest neighbors. In this question, we will measure the accuracy of a 1-nearest-neighbor classifier, i.e., predict the output as the label of the nearest neighbor in the training data. Although there are simpler ways of computing this result, we will go step-by-step here to introduce you to more concepts in nearest neighbors and SFrames, which will be useful later in this Specialization. 

**Training models**: For this question, you will need the nearest neighbors models you learned above on the training data, i.e., the dog_model, cat_model, automobile_model and bird_model.

**Spliting test data by label**: Above, you split the train data SFrame into one SFrame for images labeled ‘dog’, another for those labeled ‘cat’, etc. Now, do the same for the test data. You can call the resulting SFrames

In [40]:
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_bird = image_test[image_test['label'] == 'bird']
image_test_automobile = image_test[image_test['label'] == 'automobile']

**Finding nearest neighbors in the training set for each part of the test set**

Using this knowledge find the closest neighbor in to the dog test data using each of the trained models

In [41]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 1

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 509000  | 100         | 858.711ms    |

| Done         | 509000  | 100         | 864.123ms    |

+--------------+---------+-------------+--------------+

In [43]:
dog_cat_neighbors.head(5)

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |        33       | 36.4196077068 |  1   |
|      1      |      30606      | 38.8353268874 |  1   |
|      2      |       5545      | 36.9763410854 |  1   |
|      3      |      19631      | 34.5750072914 |  1   |
|      4      |       7493      |  34.778824791 |  1   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

**Create an SFrame with the distances from ‘dog’ test examples to the respective nearest neighbors in each class in the training data**

 The ‘distance’ column in dog_cat_neighbors above contains the distance between each ‘dog’ image in the test set and its nearest ‘cat’ image in the training set. The question we want to answer is how many of the test set ‘dog’ images are closer to a ‘dog’ in the training set than to a ‘cat’, ‘automobile’ or ‘bird’. So, next we will create an SFrame containing just these distances per data point. The goal is to create an SFrame called dog_distances with 4 columns:

i. dog_distances[‘dog-dog’] ---- storing dog_dog_neighbors[‘distance’]

ii. dog_distances[‘dog-cat’] ---- storing dog_cat_neighbors[‘distance’]

iii. dog_distances[‘dog-automobile’] ---- storing dog_automobile_neighbors[‘distance’]

iv. dog_distances[‘dog-bird’] ---- storing dog_bird_neighbors[‘distance’]

In [61]:
dog_distances = graphlab.SFrame({'dog-automobile':dog_automobile_neighbors['distance'],
                                'dog-bird':dog_bird_neighbors['distance'],
                                'dog-cat':dog_cat_neighbors['distance'],
                                'dog-dog':dog_dog_neighbors['distance'],})

In [62]:
dog_distances.head(2)

Columns:
	dog-automobile	float
	dog-bird	float
	dog-cat	float
	dog-dog	float

Rows: 2

Data:
+----------------+---------------+---------------+---------------+
| dog-automobile |    dog-bird   |    dog-cat    |    dog-dog    |
+----------------+---------------+---------------+---------------+
| 41.9579761457  | 41.7538647304 | 36.4196077068 | 33.4773590373 |
| 46.0021331807  | 41.3382958925 | 38.8353268874 | 32.8458495684 |
+----------------+---------------+---------------+---------------+
[2 rows x 4 columns]

**Computing the number of correct predictions using 1-nearest neighbors for the dog class**

Iterate line by line and compute the number of ‘dog’ test examples where the distance to the nearest ‘dog’ was lower than that to the other classes.

In [84]:
def is_dog_correct(row):
    if (row['dog-dog'] < row['dog-bird']) & (row['dog-dog'] < row['dog-cat']) & (row['dog-dog'] < row['dog-automobile']):
        return 1
    else:
        return 0

In [85]:
dog_distances['correctly_classified'] = dog_distances.apply(is_dog_correct)

In [86]:
dog_distances.head(2)

Columns:
	dog-automobile	float
	dog-bird	float
	dog-cat	float
	dog-dog	float
	correctly_classified	int

Rows: 2

Data:
+----------------+---------------+---------------+---------------+----------------------+
| dog-automobile |    dog-bird   |    dog-cat    |    dog-dog    | correctly_classified |
+----------------+---------------+---------------+---------------+----------------------+
| 41.9579761457  | 41.7538647304 | 36.4196077068 | 33.4773590373 |          1           |
| 46.0021331807  | 41.3382958925 | 38.8353268874 | 32.8458495684 |          1           |
+----------------+---------------+---------------+---------------+----------------------+
[2 rows x 5 columns]

In [87]:
dog_distances['correctly_classified'].sum()

678

In [88]:
len(dog_distances)

1000

In [89]:
accuracy = float(dog_distances['correctly_classified'].sum()) / float(len(dog_distances))

In [90]:
accuracy

0.678